<a href="https://colab.research.google.com/github/jblanco999/covidgetdata/blob/main/CovidGetData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Herramienta para analisis de datos
import pandas as pd
# Herramientas de fecha y hora
from datetime import datetime
# Autentificación
from google.colab import auth
# Google Sheets
import gspread
# mostrar el progreso
from IPython.display import HTML, display, clear_output

# Calcula los dias entre dos fechas, y los dias entre una dia cualquiera y hoy

def days_between(d1, d2):
    d1 = datetime.strptime(d1, "%Y-%m-%d")
    d2 = datetime.strptime(d2, "%Y-%m-%d")
    return abs((d2 - d1).days)

def days_to_today(d1):
   d = datetime.today()
   d2 = d.strftime('%Y-%m-%d')
   return days_between(d2,d1)

print(days_to_today("2021-03-31"))

# Adicional para mostrar en pantalla una barra de progreso

def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))




13


In [ ]:
# Obtener el dataset entero de datos Covid-19
url = 'https://sisa.msal.gov.ar/datos/descargas/covid-19/files/Covid19Casos.csv'
df = pd.read_csv(url)

In [ ]:
print(df.shape[0])

backup = df

#Filtrar columnas que no son necesarios en los datos:
df.drop("id_evento_caso", axis='columns', inplace=True)
df.drop("ultima_actualizacion", axis='columns', inplace=True)

# Eliminar todas las filas que no tengan que ver con el NOA
df = df.drop(df[df.carga_provincia_nombre == 'Buenos Aires'].index)
df = df.drop(df[df.carga_provincia_nombre == 'Córdoba'].index)
df = df.drop(df[df.carga_provincia_nombre == 'CABA'].index)
df = df.drop(df[df.carga_provincia_nombre == 'Formosa'].index)
df = df.drop(df[df.carga_provincia_nombre == 'Santa Fe'].index)
df = df.drop(df[df.carga_provincia_nombre == 'Entre Ríos'].index)
df = df.drop(df[df.carga_provincia_nombre == 'San Luis'].index)
df = df.drop(df[df.carga_provincia_nombre == 'Chaco'].index)
df = df.drop(df[df.carga_provincia_nombre == 'Corrientes'].index)
df = df.drop(df[df.carga_provincia_nombre == 'Río Negro'].index)
df = df.drop(df[df.carga_provincia_nombre == 'Santa Cruz'].index)
df = df.drop(df[df.carga_provincia_nombre == 'Neuquén'].index)
df = df.drop(df[df.carga_provincia_nombre == 'San Juan'].index)
df = df.drop(df[df.carga_provincia_nombre == 'La Pampa'].index)
df = df.drop(df[df.carga_provincia_nombre == 'Chubut'].index)
df = df.drop(df[df.carga_provincia_nombre == 'Tierra del Fuego'].index)
df = df.drop(df[df.carga_provincia_nombre == 'Misiones'].index)
df = df.drop(df[df.carga_provincia_nombre == 'La Rioja'].index)
df = df.drop(df[df.carga_provincia_nombre == 'Mendoza'].index)

# Regeneramos los índices
df.reset_index(drop=True)

8734049


,sexo,edad,edad_años_meses,residencia_pais_nombre,residencia_provincia_nombre,residencia_departamento_nombre,carga_provincia_nombre,fecha_inicio_sintomas,fecha_apertura,sepi_apertura,fecha_internacion,cuidado_intensivo,fecha_cui_intensivo,fallecido,fecha_fallecimiento,asistencia_respiratoria_mecanica,carga_provincia_id,origen_financiamiento,clasificacion,clasificacion_resumen,residencia_provincia_id,fecha_diagnostico,residencia_departamento_id
0,M,52.0,Años,Argentina,Salta,Capital,Salta,2021-04-08,2021-04-08,14,NaN,NO,NaN,NO,NaN,NO,66,Público,Caso con resultado negativo-no conclusivo - Ac...,Sospechoso,66,2021-04-08,28
1,F,34.0,Años,Argentina,Santiago del Estero,Capital,Santiago del Estero,2021-03-16,2021-04-08,14,NaN,NO,NaN,NO,NaN,NO,86,Público,Caso Descartado,Descartado,86,2021-03-18,49
2,M,2.0,Años,Argentina,Tucumán,Tafí Viejo,Tucumán,NaN,2020-06-01,23,NaN,NO,NaN,NO,NaN,NO,90,Público,Caso Descartado,Descartado,90,2020-06-06,105
3,M,40.0,Años,Argentina,Tucumán,Capital,Tucumán,NaN,2021-04-08,14,NaN,NO,NaN,NO,NaN,NO,90,Público,Caso con resultado negativo-no conclusivo - Ac...,Sospechoso,90,2021-04-07,84
4,F,36.0,Años,Argentina,Santiago del Estero,Capital,Santiago del Estero,2021-04-03,2021-04-08,14,NaN,NO,NaN,NO,NaN,NO,86,Público,Caso confirmado por laboratorio - Activo,Confirmado,86,2021-04-06,49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609369,F,36.0,Años,Argentina,Salta,Grl. José de San Martín,Salta,NaN,2021-04-08,14,NaN,NO,NaN,NO,NaN,NO,66,Público,Caso con resultado negativo-no conclusivo - Ac...,Sospechoso,66,2021-04-08,56
609370,F,67.0,Años,Argentina,Jujuy,San Pedro,Jujuy,NaN,2021-04-08,14,NaN,NO,NaN,NO,NaN,NO,38,Privado,Caso Descartado,Descartado,38,2021-04-08,63
609371,M,36.0,Años,Argentina,Salta,Capital,Salta,2021-04-06,2021-04-08,14,NaN,NO,NaN,NO,NaN,NO,66,Público,Caso confirmado por laboratorio - Activo,Confirmado,66,2021-04-08,28
609372,M,23.0,Años,Argentina,Tucumán,Tafí Viejo,Tucumán,NaN,2021-04-08,14,NaN,NO,NaN,NO,NaN,NO,90,Público,Caso confirmado por laboratorio - Activo,Confirmado,90,2021-04-07,105


In [ ]:
# Genera un dataFrame con los valores únicos de cada columna, para visualizarlos mejor y descargar la versión resumida

dfUnique = df.apply(lambda col: ', '.join(map(str, col.unique())))
print(dfUnique)
print(df.shape[0])
dfUnique.to_csv('dfUnique.csv')


sexo                                                                         M, F, NR
edad                                52.0, 34.0, 2.0, 40.0, 36.0, 26.0, 23.0, 46.0,...
edad_años_meses                                                           Años, Meses
residencia_pais_nombre              Argentina, SIN ESPECIFICAR, Uruguay, Paraguay,...
residencia_provincia_nombre         Salta, Santiago del Estero, Tucumán, Catamarca...
residencia_departamento_nombre      Capital, Tafí Viejo, Tinogasta, Lules, Yerba B...
carga_provincia_nombre              Salta, Santiago del Estero, Tucumán, Catamarca...
fecha_inicio_sintomas               2021-04-08, 2021-03-16, nan, 2021-04-03, 2021-...
fecha_apertura                      2021-04-08, 2020-06-01, 2021-04-09, 2020-06-02...
sepi_apertura                       14, 23, 15, 24, 25, 26, 27, 28, 29, 30, 31, 32...
fecha_internacion                   nan, 2021-04-07, 2021-04-06, 2021-03-31, 2021-...
cuidado_intensivo                                     

In [ ]:
# Generar nuevo dataframe con las columnas requeridas

# Primero separamos todas las fechas disponibles:
fechas = df["fecha_apertura"].unique().tolist()
fechas.extend(df["fecha_inicio_sintomas"].unique().tolist())
fechasFiltradas = []

# Elimino los valores que no son texto
for fecha in fechas:
  if isinstance(fecha,str):
    fechasFiltradas.append(fecha)

# Elimino los duplicados
fechasFiltradas = list(dict.fromkeys(fechasFiltradas))

# Las ordeno por fecha
fechasFiltradas.sort(key=days_to_today)

In [ ]:
#Generar el dataset revisado
datosRevisados = pd.DataFrame( columns = ['Fecha' , 'NOA_Confirmados', 'Tucumán', 'Salta' , 'Santiago del Estero' , 'Catamarca' , 'Jujuy']) 

for fecha in fechasFiltradas:
  data = [fecha,0,0,0,0,0,0]
  datosRevisados.loc[len(datosRevisados)] = data

totalRows = df.shape[0]
contador = 0
out = display(progress(0, totalRows), display_id=True)

for row in df.itertuples(index=True):
  contador = contador + 1
  index = datosRevisados['Fecha'].loc[lambda x: x==row.fecha_apertura].index

  if contador % 10000 == 0:
    clear_output()    
    out.update(progress(contador, totalRows))
    print(contador,"/",totalRows , "==>", ((contador/totalRows)*100),"%")

  if row.clasificacion_resumen == "Confirmado":
    datosRevisados["NOA_Confirmados"][index] = datosRevisados["NOA_Confirmados"][index] + 1
    if row.carga_provincia_nombre == "Tucumán":
      datosRevisados["Tucumán"][index] = datosRevisados["Tucumán"][index] + 1
    if row.carga_provincia_nombre == "Salta":
      datosRevisados["Salta"][index] = datosRevisados["Salta"][index] + 1
    if row.carga_provincia_nombre == "Santiago del Estero":
      datosRevisados["Santiago del Estero"][index] = datosRevisados["Santiago del Estero"][index] + 1
    if row.carga_provincia_nombre == "Catamarca":
      datosRevisados["Catamarca"][index] = datosRevisados["Catamarca"][index] + 1
    if row.carga_provincia_nombre == "Jujuy":
      datosRevisados["Jujuy"][index] = datosRevisados["Jujuy"][index] + 1

print(datosRevisados)
datosRevisados.to_csv('datosRevisados.csv')

600000 / 609374 ==> 98.46170003971288 %
          Fecha NOA_Confirmados Tucumán  ... Santiago del Estero Catamarca Jujuy
0    2021-04-12             568     344  ...                  71        83    21
1    2021-04-11             488     234  ...                 117       106    20
2    2021-04-10             699     347  ...                 180        60    65
3    2021-04-09            1127     442  ...                 182       218    49
4    2021-04-08            1442     891  ...                 167       113    60
..          ...             ...     ...  ...                 ...       ...   ...
436  2020-01-06               0       0  ...                   0         0     0
437  2020-01-05               0       0  ...                   0         0     0
438  2020-01-03               0       0  ...                   0         0     0
439  2020-01-02               0       0  ...                   0         0     0
440  2020-01-01               0       0  ...                   0     

In [ ]:
# authenticate
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials as GC
gc = gspread.authorize(GC.get_application_default())

# create, and save df
from gspread_dataframe import set_with_dataframe
dataSheet = gc.open_by_key("1aB2CY8imJ6rVOs0u-IGvFgiNtM72SsU7FKQbOQPr5zg")
sheet = dataSheet.worksheet("Contagios Tucumán - MSN")

set_with_dataframe(sheet, datosRevisados)
#set_with_dataframe(sheet, dfTuc)

